In [ ]:
# Libraries

import sys 

# Regular Expression

import re as re

# API

import requests

# Web Scrapping

from bs4 import BeautifulSoup

# Data analysis

import scipy as sp
import pandas as pd
import numpy as np

# Time Serie

import time
from datetime import date
from datetime import datetime

# JSON

from pandas.io.json import json_normalize

# Visualization

import seaborn as sns
import missingno as msno
import matplotlib as mpl
import matplotlib.pyplot as plt

# Warnings

import warnings
warnings.filterwarnings("ignore")

# Set matplolib style to seabor

mpl.style.use('seaborn')

# Pandas Settings

pd.set_option('display.width', 1000)
pd.set_option('display.precision', 3)
pd.set_option('display.max_rows', 1000)
pd.set_option("display.max_columns", None)
pd.set_option('display.colheader_justify', 'center')

# Google libraries 

from google.colab import drive

In [ ]:
# Check the version of libraries :

print('Libraries Version:')
print('SciPy : {}'. format(sp.__version__))
print('NumPy : {}'. format(np.__version__))
print('Pandas : {}'. format(pd.__version__))
print('Matplotlib : {}'. format(mpl.__version__))

Libraries Version:
SciPy : 1.7.3
NumPy : 1.21.6
Pandas : 1.3.5
Matplotlib : 3.2.2


In [ ]:
# Connect to Google Drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Part of code get from stackoverflow Expand List of Dicts inside Json file

df = pd.DataFrame()

df.reset_index(drop=True, inplace=True)

# Replace NaN by empty dict

def replace_nans_with_dict(series):
    for idx in series[series.isnull()].index:
        series.at[idx] = {}
    return series

# Explodes list and dicts

def df_explosion(df, col_name:str):

    if df[col_name].isna().any():
        df[col_name] = replace_nans_with_dict(df[col_name])

    df.reset_index(drop=True, inplace=True)

    df1 = pd.DataFrame(df.loc[:,col_name].values.tolist()).add_prefix(col_name + '_')

    df = pd.concat([df,df1], axis=1)

    df.drop([col_name], axis=1, inplace=True)

    return df

In [ ]:
# API to get list of all leagues avaliable

url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

querystring = {"season":"2016"}

headers = {
	"X-RapidAPI-Key": "932a38a4a9msh7fe8dd4a83b44d6p1cba37jsnd6667749930b",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

data = response.json()

print(response.status_code)
print(response.reason)
print(response.text)

{"get":"leagues","parameters":{"season":"2016"},"errors":[],"results":312,"paging":{"current":1,"total":1},"response":[{"league":{"id":61,"name":"Ligue 1","type":"League","logo":"https:\/\/media.api-sports.io\/football\/leagues\/61.png"},"country":{"name":"France","code":"FR","flag":"https:\/\/media.api-sports.io\/flags\/fr.svg"},"seasons":[{"year":2016,"start":"2016-08-12","end":"2017-05-20","current":false,"coverage":{"fixtures":{"events":true,"lineups":true,"statistics_fixtures":true,"statistics_players":true},"standings":true,"players":true,"top_scorers":true,"top_assists":true,"top_cards":true,"injuries":false,"predictions":true,"odds":false}}]},{"league":{"id":62,"name":"Ligue 2","type":"League","logo":"https:\/\/media.api-sports.io\/football\/leagues\/62.png"},"country":{"name":"France","code":"FR","flag":"https:\/\/media.api-sports.io\/flags\/fr.svg"},"seasons":[{"year":2016,"start":"2016-07-29","end":"2017-05-19","current":false,"coverage":{"fixtures":{"events":true,"lineups":

In [ ]:
# API get data file and transforme to dataframe

df_league = json_normalize(
    data["response"], 
    sep="_"
  )

df_league = df_league.explode('seasons')

df_league = df_explosion(df_league, 'seasons')

df_league = df_explosion(df_league, 'seasons_coverage')

df_league = df_explosion(df_league, 'seasons_coverage_fixtures')

df_league.head()


,league_id,league_name,league_type,league_logo,country_name,country_code,country_flag,seasons_year,seasons_start,seasons_end,seasons_current,seasons_coverage_standings,seasons_coverage_players,seasons_coverage_top_scorers,seasons_coverage_top_assists,seasons_coverage_top_cards,seasons_coverage_injuries,seasons_coverage_predictions,seasons_coverage_odds,seasons_coverage_fixtures_events,seasons_coverage_fixtures_lineups,seasons_coverage_fixtures_statistics_fixtures,seasons_coverage_fixtures_statistics_players
0,61,Ligue 1,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg,2016,2016-08-12,2017-05-20,False,True,True,True,True,True,False,True,False,True,True,True,True
1,62,Ligue 2,League,https://media.api-sports.io/football/leagues/6...,France,FR,https://media.api-sports.io/flags/fr.svg,2016,2016-07-29,2017-05-19,False,True,True,True,True,True,False,True,False,True,True,True,True
2,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2016,2016-06-10,2016-07-10,False,True,False,False,False,False,False,True,False,True,True,True,True
3,2,UEFA Champions League,Cup,https://media.api-sports.io/football/leagues/2...,World,None,None,2016,2016-06-28,2017-06-03,False,True,True,True,True,True,False,True,False,True,True,True,True
4,3,UEFA Europa League,Cup,https://media.api-sports.io/football/leagues/3...,World,None,None,2016,2016-06-28,2017-05-24,False,True,True,True,True,True,False,True,False,True,True,True,True


In [ ]:

df_aux = df_league.groupby(['country_name','league_type','league_name']).agg({'league_id': 'max', 'seasons_year' : ['min','max']}).reset_index()

df_aux[df_aux.country_name == 'Brazil']

country_name league_type    league_name    league_id seasons_year      
                                                 max        min       max 
52    Brazil          Cup      Copa Do Brasil      73       2016      2016
53    Brazil       League             Acreano     520       2016      2016
54    Brazil       League           Amapaense     521       2016      2016
55    Brazil       League          Amazonense     522       2016      2016
56    Brazil       League    Brasileiro Women      74       2016      2016
57    Brazil       League             Serie A      71       2016      2016
58    Brazil       League             Serie B      72       2016      2016
59    Brazil       League             Serie C      75       2016      2016
60    Brazil       League             Serie D      76       2016      2016

In [ ]:
#API

# Set the year for the 

league_cod = 71
year = 2022

# Get total number of pages 

url = "https://api-football-v1.p.rapidapi.com/v3/players"

headers = {
  "X-RapidAPI-Key": "932a38a4a9msh7fe8dd4a83b44d6p1cba37jsnd6667749930b",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

params = {
  'league' : league_cod,
  'season' : year
  }

response = requests.request("GET", url, headers=headers, params=params)



data = response.json()

nro_pages = data['paging']['total']

print(response.status_code)
print(response.reason)
print(response.text)
print(nro_pages)

200
OK
{"get":"players","parameters":{"league":"71","season":"2022"},"errors":[],"results":20,"paging":{"current":1,"total":53},"response":[{"player":{"id":9361,"name":"Henrique Vermudt","firstname":"Henrique","lastname":"Vermudt","age":23,"birth":{"date":"1999-02-25","place":"Cascavel","country":"Brazil"},"nationality":"Brazil","height":"185 cm","weight":null,"injured":false,"photo":"https:\/\/media.api-sports.io\/football\/players\/9361.png"},"statistics":[{"team":{"id":147,"name":"Coritiba","logo":"https:\/\/media.api-sports.io\/football\/teams\/147.png"},"league":{"id":71,"name":"Serie A","country":"Brazil","logo":"https:\/\/media.api-sports.io\/football\/leagues\/71.png","flag":"https:\/\/media.api-sports.io\/flags\/br.svg","season":2022},"games":{"appearences":null,"lineups":null,"minutes":null,"number":null,"position":"Midfielder","rating":null,"captain":false},"substitutes":{"in":null,"out":null,"bench":null},"shots":{"total":null,"on":null},"goals":{"total":null,"conceded":nul

In [ ]:
#API

# List to receive the API result

api_results = []

# Loop throught all pages

for pag_number in range (1,nro_pages+1):
  
  params = {
  'league' : league_cod,
  'season' : year,
  'page' : pag_number
  }
  
  response = requests.request("GET", url, headers=headers, params=params)
  
  data = response.json()
  
  api_results.append(data)

  time.sleep(6)       

In [ ]:
# Check the number to result stored

print(len(api_results))
for i in api_results:
  print(i)

53
{'get': 'players', 'parameters': {'league': '71', 'page': '1', 'season': '2022'}, 'errors': [], 'results': 20, 'paging': {'current': 1, 'total': 53}, 'response': [{'player': {'id': 9361, 'name': 'Henrique Vermudt', 'firstname': 'Henrique', 'lastname': 'Vermudt', 'age': 23, 'birth': {'date': '1999-02-25', 'place': 'Cascavel', 'country': 'Brazil'}, 'nationality': 'Brazil', 'height': '185 cm', 'weight': None, 'injured': False, 'photo': 'https://media.api-sports.io/football/players/9361.png'}, 'statistics': [{'team': {'id': 147, 'name': 'Coritiba', 'logo': 'https://media.api-sports.io/football/teams/147.png'}, 'league': {'id': 71, 'name': 'Serie A', 'country': 'Brazil', 'logo': 'https://media.api-sports.io/football/leagues/71.png', 'flag': 'https://media.api-sports.io/flags/br.svg', 'season': 2022}, 'games': {'appearences': None, 'lineups': None, 'minutes': None, 'number': None, 'position': 'Midfielder', 'rating': None, 'captain': False}, 'substitutes': {'in': None, 'out': None, 'bench'

In [ ]:
# Tranform list result to Dataframe

df_list = []

for i in range(0,len(api_results)) : 
  
  df_aux = json_normalize(api_results[i]["response"], sep="_")
  
  df_aux['Paging'] = api_results[i]["paging"]['current']
  
  df_list.append(df_aux)

df = pd.concat(df_list)

print(len(df))

df.head()


1048


,statistics,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,player_weight,player_injured,player_photo,Paging
0,"[{'team': {'id': 147, 'name': 'Coritiba', 'log...",9361,Henrique Vermudt,Henrique,Vermudt,23,1999-02-25,Cascavel,Brazil,Brazil,185 cm,None,False,https://media.api-sports.io/football/players/9...,1
1,"[{'team': {'id': 134, 'name': 'Atletico Parana...",9650,Reinaldo,Reinaldo,Nascimento Satorno,21,2001-06-06,Siderópolis,Brazil,Brazil,177 cm,77 kg,False,https://media.api-sports.io/football/players/9...,1
2,"[{'team': {'id': 134, 'name': 'Atletico Parana...",136124,Edu,Lucas Eduardo,Ribeiro de Souza,22,2000-06-16,Belo Horizonte,Brazil,Brazil,188 cm,87 kg,False,https://media.api-sports.io/football/players/1...,1
3,"[{'team': {'id': 120, 'name': 'Botafogo', 'log...",197323,Wendel Lessa,Wendel,Lessa de Macedo Gonçalves,20,2002-02-06,None,Brazil,Brazil,None,None,False,https://media.api-sports.io/football/players/1...,1
4,"[{'team': {'id': 1193, 'name': 'Cuiaba', 'logo...",208808,Marcos Brito,Marcos,Rodrigues de Brito,21,2001-01-30,None,Brazil,Brazil,None,None,False,https://media.api-sports.io/football/players/2...,1


In [ ]:
# Normalize the dataframe

df = df.explode('statistics')

df = df_explosion(df, 'statistics')

df = df_explosion(df, 'statistics_team')

df = df_explosion(df, 'statistics_league')

df = df_explosion(df, 'statistics_games')

df = df_explosion(df, 'statistics_substitutes')

df = df_explosion(df, 'statistics_shots')

df = df_explosion(df, 'statistics_goals')

df = df_explosion(df, 'statistics_passes')

df = df_explosion(df, 'statistics_tackles')

df = df_explosion(df, 'statistics_duels')

df = df_explosion(df, 'statistics_dribbles')

df = df_explosion(df, 'statistics_fouls')

df = df_explosion(df, 'statistics_cards')

df = df_explosion(df, 'statistics_penalty')

df.head()

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,player_weight,player_injured,player_photo,Paging,statistics_team_id,statistics_team_name,statistics_team_logo,statistics_league_id,statistics_league_name,statistics_league_country,statistics_league_logo,statistics_league_flag,statistics_league_season,statistics_games_appearences,statistics_games_lineups,statistics_games_minutes,statistics_games_number,statistics_games_position,statistics_games_rating,statistics_games_captain,statistics_substitutes_in,statistics_substitutes_out,statistics_substitutes_bench,statistics_shots_total,statistics_shots_on,statistics_goals_total,statistics_goals_conceded,statistics_goals_assists,statistics_goals_saves,statistics_passes_total,statistics_passes_key,statistics_passes_accuracy,statistics_tackles_total,statistics_tackles_blocks,statistics_tackles_interceptions,statistics_duels_total,statistics_duels_won,statistics_dribbles_attempts,statistics_dribbles_success,statistics_dribbles_past,statistics_fouls_drawn,statistics_fouls_committed,statistics_cards_yellow,statistics_cards_yellowred,statistics_cards_red,statistics_penalty_won,statistics_penalty_commited,statistics_penalty_scored,statistics_penalty_missed,statistics_penalty_saved
0,9361,Henrique Vermudt,Henrique,Vermudt,23,1999-02-25,Cascavel,Brazil,Brazil,185 cm,None,False,https://media.api-sports.io/football/players/9...,1,147,Coritiba,https://media.api-sports.io/football/teams/147...,71,Serie A,Brazil,https://media.api-sports.io/football/leagues/7...,https://media.api-sports.io/flags/br.svg,2022,NaN,NaN,NaN,None,Midfielder,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
1,9650,Reinaldo,Reinaldo,Nascimento Satorno,21,2001-06-06,Siderópolis,Brazil,Brazil,177 cm,77 kg,False,https://media.api-sports.io/football/players/9...,1,134,Atletico Paranaense,https://media.api-sports.io/football/teams/134...,71,Serie A,Brazil,https://media.api-sports.io/football/leagues/7...,https://media.api-sports.io/flags/br.svg,2022,NaN,NaN,NaN,None,Attacker,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,136124,Edu,Lucas Eduardo,Ribeiro de Souza,22,2000-06-16,Belo Horizonte,Brazil,Brazil,188 cm,87 kg,False,https://media.api-sports.io/football/players/1...,1,134,Atletico Paranaense,https://media.api-sports.io/football/teams/134...,71,Serie A,Brazil,https://media.api-sports.io/football/leagues/7...,https://media.api-sports.io/flags/br.svg,2022,NaN,NaN,NaN,None,Defender,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,197323,Wendel Lessa,Wendel,Lessa de Macedo Gonçalves,20,2002-02-06,None,Brazil,Brazil,None,None,False,https://media.api-sports.io/football/players/1...,1,120,Botafogo,https://media.api-sports.io/football/teams/120...,71,Serie A,Brazil,https://media.api-sports.io/football/leagues/7...,https://media.api-sports.io/flags/br.svg,2022,NaN,NaN,NaN,None,Midfielder,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
4,208808,Marcos Brito,Marcos,Rodrigues de Brito,21,2001-01-30,None,Brazil,Brazil,None,None,False,https://media.api-sports.io/football/players/2...,1,1193,Cuiaba,https://media.api-sports.io/football/teams/119...,71,Serie A,Brazil,https://media.api-sports.io/football/leagues/7...,https://media.api-sports.io/flags/br.svg,2022,NaN,NaN,NaN,None,Midfielder,None,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN


In [ ]:
# Export dataframe to csv file 

file_path = '/content/drive/My Drive/'
file_name = 'Serie_A_League_Statistics_' + str(year) + '.csv'

df.to_csv(file_path + file_name, index=False)